> *Processo de ingestão da Tabela XPTO da API do IBGE SIDRA*

| Camada      | Entidade              |
| ----------  | ----------            |
| raw         | pib_ibge.json      |



> *Histórico de Alterações*

| Time      | Projeto               | Autor                                   | Data               | Descrição                                                              |
|-----------|-----------------------|-----------------------------------------|--------------------|------------------------------------------------------------------------|
| AL DataLab       | Data Engineer Challenge     | Ana/Lucas                     |   08/12/2025       | Desenvolvimento inicial do notebook                                    |
| AL DataLab       | Data Engineer Challenge     | xxxxxxxxxxxxxxxxx                       |   xx/xx/xxxx       | xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx                                     |

In [0]:
file_name   = 'ibge_pib'
extension   ='json'

url_api= 'https://apisidra.ibge.gov.br/values/t/5938/n3/all/v/37/p/all/f/u'

In [0]:
%run "/Workspace/Users/al.datalab.global@gmail.com/desafio_ibge/04 - UTILS/import_libs"

In [0]:
# DEFINE A VARIÁVEL DT_CARGA
dt_carga = datetime.now().strftime('%Y-%m-%d')

# CONSTRÓI O PATH PARA SALVAR O ARQUIVO NA CAMADA RAW
base_path   = f'/Volumes/dataengineerchallenge/1_raw/raw_volume'
dir_path    = f"{base_path}/{dt_carga}"
file_path   = f"{dir_path}/{file_name}.{extension}"

# CRIA O DIRETÓRIO BASEADO NO PATH
dbutils.fs.mkdirs(dir_path)

display(f"O caminho para o volume de dados é {file_path}")

In [0]:
retry_strategy = Retry(
    total=4,
    status_forcelist=[429, 500, 502, 503, 504],
) # Usado para tentar a conexão novamente quando a API do IBGE falhar 
adapter = HTTPAdapter (max_retries=retry_strategy) 
session = requests.Session() 
session.mount(url_api, adapter)
response = session.get(url_api)

response.raise_for_status()

dados = response.json()
#Session define as regras e o adapter implementa as regras no momento da requisição

In [0]:
# Serializar em JSON
json_str = json.dumps(dados, ensure_ascii=False)

In [0]:
with open(file_path, "w", encoding="utf-8") as f:
    f.write(json_str)






**adapter = HTTPAdapter (max_retries=retry_strategy)**
Cria um adaptador HTTP responsável por gerenciar 

as conexões. Aqui eu adiciono a estratégia de retry (quantas tentativas fazer, para quais erros, etc).
Esse adaptador será acoplado à Session para controlar todas as requisições.

**session = requests.Session()** Session melhora performance de rede reutilizando o mesmo canal de comunicação e permite configurar adaptadores (como retry). Todas as requisições enviadas pela Session usarão essas configurações.

**session.mount(url_api_pib, adapter)** Associa o adaptador (com retry) a essa URL.
O método .mount() diz à Session para usar esse adaptador para qualquer requisição cujo prefixo combine com esse.
Ou seja, a partir daqui, toda requisição para o domínio configurado usará automaticamente a política de retry, pool de conexões e controle do adaptador.

**r = session.get(url_api_pib)** Envia uma requisição GET para a API usando a Session configurada com retry e adaptador.

